### All needed imports

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfdata

### Loading Data

In [3]:
mnist_data, mnist_info = tfdata.load(name='mnist', with_info=True, as_supervised=True, try_gcs=True)

#### MNIST images contain images in shape 28x28 with values from 0 to 255

In [4]:
mnist_data, mnist_info

({'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
  'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>},
 tfds.core.DatasetInfo(
     name='mnist',
     version=3.0.1,
     description='The MNIST database of handwritten digits.',
     homepage='http://yann.lecun.com/exdb/mnist/',
     features=FeaturesDict({
         'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
         'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
     }),
     total_num_examples=70000,
     splits={
         'test': 10000,
         'train': 60000,
     },
     supervised_keys=('image', 'label'),
     citation="""@article{lecun2010mnist,
       title={MNIST handwritten digit database},
       author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
       journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
       volume={2},
       year={2010}
     }""",
     redistribution_info=,
 ))

In [6]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

#### Making validation set 10% of training set

In [13]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

### Scaling the Data to make result numerically stable
#### Inputs between 0 and 1

In [20]:
def scale(img, label):
    # we make sure the value is a float
    img = tf.cast(img, tf.float32) 
    img /= 255.

    return img, label

#### Apllying custom transformation 
##### <i>Map</i> only takes inputs and labels and returns image and label

In [21]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [22]:
test_data = mnist_test.map(scale)

### Shuffling Data and creating validation dataset

In [23]:
buffer = 10000

In [24]:
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer)

### Creating validation set and then training set without validation

In [25]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

In [26]:
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

##### Should've used train_test_split

#### Batch size

In [27]:
batch = 100

##### <i>batch</i>-method combines the consecutive elements ad dataset into batches

In [28]:
train_data = train_data.batch(BATCH_SIZE)

#### We don't need to specify batches for valid set, but Model expects the validation data in batches

In [29]:
validation_data = validation_data.batch(num_validation_samples)

In [30]:
test_data = test_data.batch(num_test_samples)

#### Validation data must be in the same format as train/test

In [31]:
validation_inputs, validation_targets = next(iter(validation_data))

## Model

It is going to habe 28x28 ( = 784) input nodes, 2 hidden layers with 50 nodes each and output layer with 10 nodes (for Softmax)

In [32]:
input_size = 784
output_size = 10
hidden_units = 50

#### Since, I won't use CNN, I must flatten images into a vector (Flatten layer )

In [33]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    tf.keras.layers.Dense(hidden_units, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_units, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [34]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
# determine the maximum number of epochs
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets),validation_steps=10)

Epoch 1/5
540/540 [==============================] - 3s 5ms/step - loss: 0.4228 - accuracy: 0.8784 - val_loss: 0.2081 - val_accuracy: 0.9393
Epoch 2/5
540/540 [==============================] - 2s 3ms/step - loss: 0.1954 - accuracy: 0.9427 - val_loss: 0.1575 - val_accuracy: 0.9540
Epoch 3/5
540/540 [==============================] - 2s 3ms/step - loss: 0.1513 - accuracy: 0.9561 - val_loss: 0.1287 - val_accuracy: 0.9645
Epoch 4/5
540/540 [==============================] - 2s 3ms/step - loss: 0.1260 - accuracy: 0.9632 - val_loss: 0.1148 - val_accuracy: 0.9662
Epoch 5/5
540/540 [==============================] - 2s 3ms/step - loss: 0.1054 - accuracy: 0.9693 - val_loss: 0.1047 - val_accuracy: 0.9707


### Testing

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                39250     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.1209 - accuracy: 0.9643

[0.12085266411304474, 0.9643]